In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/nyc311_noise_residential_brooklyn_2023.csv")


In [3]:
# 0) Базовая инфа
print("shape:", df.shape)
print("\ncolumns:", list(df.columns))

# 1) Пропуски (NaN)
na = df.isna().sum().sort_values(ascending=False)
print("\nNaN per column (top):")
print(na[na > 0].head(30))

shape: (86040, 16)

columns: ['unique_key', 'created_date', 'closed_date', 'complaint_type', 'descriptor', 'location_type', 'incident_address', 'street_name', 'incident_zip', 'borough', 'city', 'latitude', 'longitude', 'agency', 'status', 'resolution_description']

NaN per column (top):
city                      1340
latitude                   285
longitude                  285
resolution_description       4
incident_zip                 3
dtype: int64


In [4]:
# 2) Пустые строки / 'NA' / 'null' как текст
def is_blankish(s: pd.Series) -> pd.Series:
    x = s.astype("string")
    x = x.str.strip()
    return x.isna() | (x == "") | x.str.lower().isin(["na", "n/a", "null", "none", "nan"])

blankish_counts = {}
for c in df.columns:
    if df[c].dtype == "object" or str(df[c].dtype).startswith("string"):
        blankish_counts[c] = int(is_blankish(df[c]).sum())
blankish = pd.Series(blankish_counts).sort_values(ascending=False)
print("\nBlank-ish text values per column (top):")
print(blankish[blankish > 0].head(30))


Blank-ish text values per column (top):
city                      1340
resolution_description       4
dtype: int64


In [5]:
# 3) Дубликаты unique_key
if "unique_key" in df.columns:
    dup_keys = df["unique_key"].duplicated().sum()
    print("\nDuplicate unique_key rows:", int(dup_keys))


Duplicate unique_key rows: 0


In [6]:
# 4) Грязные даты + невозможные длительности
for col in ["created_date", "closed_date"]:
    if col in df.columns:
        df[col + "_dt"] = pd.to_datetime(df[col], errors="coerce", utc=True)

bad_created = int(df["created_date_dt"].isna().sum()) if "created_date_dt" in df.columns else 0
bad_closed  = int(df["closed_date_dt"].isna().sum()) if "closed_date_dt" in df.columns else 0
print("\nBad datetime parse:")
print("  created_date bad:", bad_created)
print("  closed_date  bad:", bad_closed)

if "created_date_dt" in df.columns and "closed_date_dt" in df.columns:
    dur = (df["closed_date_dt"] - df["created_date_dt"]).dt.total_seconds() / 3600.0
    # закрыто раньше, чем создано
    neg = int((dur < 0).sum())
    # "вечные" заявки (закрытия нет)
    open_cnt = int(df["closed_date_dt"].isna().sum())
    # подозрительно долгие (например, > 30 дней)
    long30 = int((dur > 24*30).sum())
    print("\nDuration sanity (hours):")
    print("  open (no closed_date):", open_cnt)
    print("  negative duration:", neg)
    print("  >30 days:", long30)


Bad datetime parse:
  created_date bad: 0
  closed_date  bad: 0

Duration sanity (hours):
  open (no closed_date): 0
  negative duration: 9
  >30 days: 2


In [7]:
# 5) Координаты: пустые/нечисловые/вне диапазона
def to_num(s):
    return pd.to_numeric(s, errors="coerce")

if "latitude" in df.columns:
    lat = to_num(df["latitude"])
    bad_lat = int(lat.isna().sum() + ((lat < -90) | (lat > 90)).sum())
    print("\nLatitude bad count (NaN or out of range):", bad_lat)

if "longitude" in df.columns:
    lon = to_num(df["longitude"])
    bad_lon = int(lon.isna().sum() + ((lon < -180) | (lon > 180)).sum())
    print("Longitude bad count (NaN or out of range):", bad_lon)

# 6) Быстрый взгляд на распределение категорий (чтобы увидеть мусорные значения)
for c in ["borough", "incident_zip", "agency", "status", "descriptor", "location_type"]:
    if c in df.columns:
        print(f"\nTop values: {c}")
        print(df[c].astype("string").str.strip().value_counts(dropna=False).head(10))


Latitude bad count (NaN or out of range): 285
Longitude bad count (NaN or out of range): 285

Top values: borough
borough
BROOKLYN    86040
Name: count, dtype: Int64

Top values: incident_zip
incident_zip
11226.0    5581
11221.0    5461
11207.0    4663
11212.0    4167
11208.0    3875
11201.0    3769
11233.0    3729
11206.0    3363
11216.0    2929
11213.0    2890
Name: count, dtype: Int64

Top values: agency
agency
NYPD    86040
Name: count, dtype: Int64

Top values: status
status
Closed         86038
Unspecified        2
Name: count, dtype: Int64

Top values: descriptor
descriptor
Loud Music/Party    48441
Banging/Pounding    29746
Loud Talking         6590
Loud Television      1263
Name: count, dtype: Int64

Top values: location_type
location_type
Residential Building/House    86040
Name: count, dtype: Int64


In [8]:
import pandas as pd

# df должен уже существовать в ноутбуке (после твоего merge/подготовки).
# Если df ещё нет — НЕ продолжай ниже, сначала доведи до df.

print("rows:", len(df))
print("cols:", len(df.columns))
display(pd.DataFrame({"col": df.columns, "dtype": [str(df[c].dtype) for c in df.columns]}))
display(df.head(3))


rows: 86040
cols: 18


,col,dtype
0,unique_key,int64
1,created_date,object
2,closed_date,object
3,complaint_type,object
4,descriptor,object
5,location_type,object
6,incident_address,object
7,street_name,object
8,incident_zip,float64
9,borough,object


,unique_key,created_date,closed_date,complaint_type,descriptor,location_type,incident_address,street_name,incident_zip,borough,city,latitude,longitude,agency,status,resolution_description,created_date_dt,closed_date_dt
0,56418136,2023-01-01T00:00:46.000,2023-01-01T01:01:43.000,Noise - Residential,Loud Music/Party,Residential Building/House,1621 EAST 51 STREET,EAST 51 STREET,11234.0,BROOKLYN,BROOKLYN,40.620665,-73.926040,NYPD,Closed,The Police Department responded to the complai...,2023-01-01 00:00:46+00:00,2023-01-01 01:01:43+00:00
1,56413156,2023-01-01T00:06:00.000,2023-01-01T02:01:55.000,Noise - Residential,Loud Music/Party,Residential Building/House,649 VERMONT STREET,VERMONT STREET,11207.0,BROOKLYN,BROOKLYN,40.663259,-73.891653,NYPD,Closed,The Police Department responded to the complai...,2023-01-01 00:06:00+00:00,2023-01-01 02:01:55+00:00
2,56417885,2023-01-01T00:06:28.000,2023-01-01T01:27:32.000,Noise - Residential,Loud Music/Party,Residential Building/House,2078 CROPSEY AVENUE,CROPSEY AVENUE,11214.0,BROOKLYN,BROOKLYN,40.598980,-74.001051,NYPD,Closed,The Police Department responded to the complai...,2023-01-01 00:06:28+00:00,2023-01-01 01:27:32+00:00


In [9]:
import re

cols = list(df.columns)

def find_candidates(patterns):
    out = []
    for c in cols:
        s = str(c).lower()
        if any(re.search(p, s) for p in patterns):
            out.append(c)
    return out

dt_candidates = find_candidates([r"created", r"date", r"datetime", r"timestamp", r"hour"])
temp_candidates = find_candidates([r"temp", r"\btmp\b", r"temperature"])
id_candidates = find_candidates([r"unique", r"complaint", r"key", r"id", r"sr[_\s]*number", r"ticket"])

print("datetime candidates:", dt_candidates)
print("temp candidates:", temp_candidates)
print("id candidates:", id_candidates)


datetime candidates: ['created_date', 'closed_date', 'created_date_dt', 'closed_date_dt']
temp candidates: []
id candidates: ['unique_key', 'complaint_type', 'incident_address', 'incident_zip']


In [10]:
# --- OPTIONAL MANUAL OVERRIDES ---
# Если хочешь — впиши точные имена колонок и оставь остальное как есть.
CREATED_COL = None      # например: "created_date" или "created_dt_local"
TEMP_COL = None         # например: "temperature_c" или "tmp_c"
ID_COL = None           # например: "unique_key" или "complaint_id"

# --- AUTO SELECT (if overrides are None) ---
def pick_first(cands):
    return cands[0] if cands else None

if CREATED_COL is None:
    # приоритет: колонки, где явно есть "hour" или "created"
    prefer = [c for c in dt_candidates if "hour" in str(c).lower()] + [c for c in dt_candidates if "created" in str(c).lower()]
    CREATED_COL = pick_first(prefer) or pick_first(dt_candidates)

if TEMP_COL is None:
    prefer = [c for c in temp_candidates if "temp" in str(c).lower()] + [c for c in temp_candidates if "tmp" in str(c).lower()]
    TEMP_COL = pick_first(prefer) or pick_first(temp_candidates)

if ID_COL is None:
    prefer = [c for c in id_candidates if "unique" in str(c).lower()] + [c for c in id_candidates if "complaint" in str(c).lower()]
    ID_COL = pick_first(prefer) or pick_first(id_candidates)

print("SELECTED:")
print("  CREATED_COL:", CREATED_COL)
print("  TEMP_COL   :", TEMP_COL)
print("  ID_COL     :", ID_COL)


SELECTED:
  CREATED_COL: created_date
  TEMP_COL   : None
  ID_COL     : unique_key


In [11]:
import pandas as pd
from pathlib import Path

# --- INPUTS (канон проекта) ---
NYC_CREATED_COL = "created_date"
NYC_ID_COL = "unique_key"

WEATHER_PATH = "data/weather_kjfk_hourly_2023.csv"

# --- 1) 311: created_hour ---
df2 = df.copy()
df2["created_hour"] = pd.to_datetime(df2[NYC_CREATED_COL], errors="coerce").dt.floor("h")

assert df2["created_hour"].isna().sum() == 0, "311 created_hour has NaT — check created_date parsing"

# --- 2) Weather: detect hour + temp columns robustly ---
w = pd.read_csv(WEATHER_PATH)

w_cols = list(w.columns)

# hour column: pick first column containing "hour" or "time" or "datetime"
hour_candidates = [c for c in w_cols if any(k in str(c).lower() for k in ["created_hour", "hour", "time", "datetime", "date"])]
temp_candidates = [c for c in w_cols if any(k in str(c).lower() for k in ["temperature_c", "temp", "tmp"])]

print("Weather hour candidates:", hour_candidates)
print("Weather temp candidates:", temp_candidates)

assert len(hour_candidates) > 0, "Cannot find weather hour column"
assert len(temp_candidates) > 0, "Cannot find weather temperature column"

WEATHER_HOUR_COL = hour_candidates[0]
# prefer exact "temperature_c" if present
WEATHER_TEMP_COL = "temperature_c" if "temperature_c" in w_cols else temp_candidates[0]

w2 = w.copy()
w2["created_hour"] = pd.to_datetime(w2[WEATHER_HOUR_COL], errors="coerce").dt.floor("h")
w2["temperature_c"] = pd.to_numeric(w2[WEATHER_TEMP_COL], errors="coerce")

w2 = (
    w2[["created_hour", "temperature_c"]]
    .dropna(subset=["created_hour"])
    .drop_duplicates(subset=["created_hour"])
)

# --- 3) Merge (many complaints per hour -> one weather record per hour) ---
df_merged = df2.merge(w2, how="left", on="created_hour", validate="m:1")

temp_null_rate = float(df_merged["temperature_c"].isna().mean())
print("Merged rows:", len(df_merged))
print("temperature_c null rate:", round(temp_null_rate, 6))

# --- 4) Canonical DF ---
df_canonical = pd.DataFrame({
    "created_hour": df_merged["created_hour"],
    "temperature_c": df_merged["temperature_c"],
    "complaint_id": df_merged[NYC_ID_COL].astype("string"),
})

assert df_canonical["created_hour"].isna().sum() == 0
assert df_canonical["complaint_id"].isna().sum() == 0

display(df_canonical.head(3))

# --- 5) Save ---
out_dir = Path("data")
out_dir.mkdir(parents=True, exist_ok=True)

pkl_path = out_dir / "nyc311_noise_brooklyn_2023_with_weather_canonical.pkl"
csv_path = out_dir / "nyc311_noise_brooklyn_2023_with_weather_canonical.csv"

df_canonical.to_pickle(pkl_path)
df_canonical.to_csv(csv_path, index=False)

print("Saved:")
print(" ", pkl_path.as_posix())
print(" ", csv_path.as_posix())


Weather hour candidates: ['hour']
Weather temp candidates: ['air_temp_c']
Merged rows: 86040
temperature_c null rate: 0.000965


,created_hour,temperature_c,complaint_id
0,2023-01-01,8.9,56418136
1,2023-01-01,8.9,56413156
2,2023-01-01,8.9,56417885


Saved:
  data/nyc311_noise_brooklyn_2023_with_weather_canonical.pkl
  data/nyc311_noise_brooklyn_2023_with_weather_canonical.csv


In [12]:
from pathlib import Path

out = Path("data/nyc311_noise_brooklyn_2023_with_weather_canonical.csv")
out.parent.mkdir(parents=True, exist_ok=True)

df_canonical.to_csv(out, index=False)
print("saved:", out.as_posix(), "rows:", len(df_canonical))


saved: data/nyc311_noise_brooklyn_2023_with_weather_canonical.csv rows: 86040
